ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'ace_tools'


False

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# Load the new CSV file
data = pd.read_csv('/content/df_with_ucla - df_with_ucla.csv')

data['Average_Coherency'] = data[['Coherency_1', 'Coherency_2', 'Coherency_3', 'Coherency_4']].mean(axis=1)
data['Average_Generation'] = data[['Generation_1', 'Generation_2', 'Generation_3', 'Generation_4']].mean(axis=1)
data['Average_Knowledge'] = data[['Knowledge_1', 'Knowledge_1', 'Knowledge_1', 'Knowledge_14']].mean(axis=1)


t_stat_coherency, p_value_coherency = ttest_rel(
    data[data['model'] == 'gpt-3.5-turbo']['Average_Coherency'],
    data[data['model'] == 'personal chatbot']['Average_Coherency']
)

# Generation
t_stat_generation, p_value_generation = ttest_rel(
    data[data['model'] == 'gpt-3.5-turbo']['Average_Generation'],
    data[data['model'] == 'personal chatbot']['Average_Generation']
)

# Knowledge
t_stat_knowledge, p_value_knowledge = ttest_rel(
    data[data['model'] == 'gpt-3.5-turbo']['Average_Knowledge'],
    data[data['model'] == 'personal chatbot']['Average_Knowledge']
)

(t_stat_coherency, p_value_coherency), (t_stat_generation, p_value_generation), (t_stat_knowledge, p_value_knowledge)

((-27.10590267672043, 3.7222500550102174e-31),
 (-6.705040236691033, 1.9002840549137604e-08),
 (-1.808881632873675, 0.07660693415560435))

In [ ]:
def run_t_test(length):
  t_stat_coherency, p_value_coherency = ttest_rel(
      data[(data['model'] == 'gpt-3.5-turbo') & (data['conversation_length'] == length)]['Average_Coherency'],
      data[(data['model'] == 'personal chatbot') & (data['conversation_length'] == length)]['Average_Coherency']
  )

  # Generation
  t_stat_generation, p_value_generation = ttest_rel(
      data[(data['model'] == 'gpt-3.5-turbo') & (data['conversation_length'] == length )]['Average_Generation'],
      data[(data['model'] == 'personal chatbot') & (data['conversation_length'] == length)]['Average_Generation']
  )

  # Knowledge
  t_stat_knowledge, p_value_knowledge = ttest_rel(
      data[(data['model'] == 'gpt-3.5-turbo') & (data['conversation_length'] == length)]['Average_Knowledge'],
      data[(data['model'] == 'personal chatbot') & (data['conversation_length'] == length)]['Average_Knowledge']
  )

  return((t_stat_coherency, p_value_coherency), (t_stat_generation, p_value_generation), (t_stat_knowledge, p_value_knowledge))


results = {
    "Length": [],

    "Coherency p-value": [],


    "Knowledge p-value": []
}

for length in [1, 3, 5]:
    coherency, generation, knowledge = run_t_test(length)
    results["Length"].append(length)

    results["Coherency p-value"].append(coherency[1])

    results["Knowledge p-value"].append(knowledge[1])

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Length,Coherency p-value,Knowledge p-value
0,1,6.034459e-12,0.433321
1,3,1.001538e-15,0.083240
2,5,4.873795e-08,0.001054


In [ ]:
# Calculate the average scores for Coherency and Knowledge
data['Average_Coherency'] = data[['Coherency_1', 'Coherency_2', 'Coherency_3', 'Coherency_4']].mean(axis=1)
data['Average_Knowledge'] = data[['Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_14']].mean(axis=1)

# Calculate the means for each length and model combination
summary = data.groupby(['conversation_length', 'model'])[['Average_Coherency', 'Average_Knowledge']].mean().unstack()

# Prepare the table with the required format
summary_table = summary[['Average_Coherency', 'Average_Knowledge']]
summary_table.columns = ['GPT3.5 Coherency', 'Persona Chatbot Coherency', 'GPT3.5 Knowledge', 'Persona Chatbot Knowledge']

summary_table


,GPT3.5 Coherency,Persona Chatbot Coherency,GPT3.5 Knowledge,Persona Chatbot Knowledge
conversation_length,,,,
1,3.3750,4.9125,3.8250,3.8000
3,3.1625,4.8750,3.5125,3.9625
5,2.6250,4.7750,3.1750,4.2250


In [ ]:
import numpy as np
# Calculate the correlation matrix for the Coherency columns
coherency_columns = ['Coherency_1', 'Coherency_2', 'Coherency_3', 'Coherency_4']
coherency_correlation = data[coherency_columns].corr()

coherency_correlation

# Extract the off-diagonal values from the correlation matrix
off_diagonal_values = coherency_correlation.values[np.triu_indices_from(coherency_correlation, k=1)]
average_correlation_coherency = off_diagonal_values.mean()

print(average_correlation_coherency)

# Calculate the correlation matrix for the Coherency columns
coherency_columns = ['Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_14']
coherency_correlation = data[coherency_columns].corr()

coherency_correlation

# Extract the off-diagonal values from the correlation matrix
off_diagonal_values = coherency_correlation.values[np.triu_indices_from(coherency_correlation, k=1)]
average_correlation_knowledge = off_diagonal_values.mean()

average_correlation_knowledge

correlation_table = pd.DataFrame({
    'Metric': ['Coherency', 'Knowledge'],
    'Average Correlation': [average_correlation_coherency, average_correlation_knowledge]
})

correlation_table


0.8584930450773961


,Metric,Average Correlation
0,Coherency,0.858493
1,Knowledge,0.715738


In [ ]:
perp_data = pd.read_csv('/content/preplexity_with_ucla.csv')

merged_data = pd.concat([data, perp_data], axis=1)

In [ ]:
def run_t_test(data, length):
  # Knowledge
  t_stat_knowledge, p_value_knowledge = ttest_rel(
      data[(data['model'] == 'gpt-3.5-turbo') & (data['conversation_length'] == length)]['perplexity_values'],
      data[(data['model'] == 'personal chatbot') & (data['conversation_length'] == length)]['perplexity_values']
  )

  return p_value_knowledge


results = {
    "Length": [],

    "Perplexity score p-value": []
}

for length in [1, 3, 5]:
  p_value_knowledge = run_t_test(merged_data, length)
  results["Length"].append(length)
  results["Perplexity score p-value"].append(p_value_knowledge)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Length,Perplexity score p-value
0,1,0.005318
1,3,0.002118
2,5,0.000140


In [ ]:
summary = merged_data.groupby(['conversation_length', 'model'])[['perplexity_values']].mean().unstack()

# Prepare the table with the required format
summary_table = summary[['perplexity_values']]
summary_table.columns = ['GPT3.5 perplexity_values', 'Persona Chatbot perplexity_values']

summary_table


,GPT3.5 perplexity_values,Persona Chatbot perplexity_values
conversation_length,,
1,1.768625,1.026043
3,2.104652,1.067653
5,1.827122,1.027634
